Import the necessary Libraries

In [ ]:
!pip install python-dotenv
!pip install supabase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import pandas as pd
import requests
import uuid
import re
from datetime import datetime
import os
from google.colab import drive
from dotenv import load_dotenv
from supabase import create_client, Client, ClientOptions


Step 1: Parse Twillio File.

In [ ]:
import pandas as pd

# Load the CSV file
file_path = '/content/sample_data/sms-log-AC36f8c891aaad24e1532861402eff88ba_2024-12-16.csv'  # Replace with your actual file path
sms_data = pd.read_csv(file_path)

# Ensure required columns exist
required_columns = {'Status', 'Direction', 'To', 'From', 'Body', 'SentDate'}
missing_columns = required_columns - set(sms_data.columns)
if missing_columns:
    raise ValueError(f"Missing columns in CSV: {missing_columns}")

# Filter delivered (outbound) and received (inbound) messages
delivered_sms_data = sms_data[(sms_data['Status'] == 'delivered') & (sms_data['Direction'] == 'outbound-api')].copy()
received_sms_data = sms_data[(sms_data['Status'] == 'received') & (sms_data['Direction'] == 'inbound')].copy()

# Convert SentDate to datetime
delivered_sms_data['SentDate'] = pd.to_datetime(delivered_sms_data['SentDate'], errors='coerce')
received_sms_data['SentDate'] = pd.to_datetime(received_sms_data['SentDate'], errors='coerce')

# Drop rows where SentDate is NaT (invalid dates)
delivered_sms_data = delivered_sms_data.dropna(subset=['SentDate']).sort_values(by='SentDate')
received_sms_data = received_sms_data.dropna(subset=['SentDate']).sort_values(by='SentDate')

# Extract unique question-answer pairs
qa_responses = []
unique_questions_by_number = {}  # Dictionary to track questions by phone number

for i, question in delivered_sms_data.iterrows():
    question_to = question['To']
    question_body = question['Body']
    delivered_sent_date = question['SentDate']

    # Initialize tracking for this phone number if not already done
    if question_to not in unique_questions_by_number:
        unique_questions_by_number[question_to] = set()

    # Skip duplicate questions for this phone number
    if question_body in unique_questions_by_number[question_to]:
        continue
    unique_questions_by_number[question_to].add(question_body)

    # Find responses after the question was sent
    matching_responses = received_sms_data[
        (received_sms_data['From'] == question_to) &
        (received_sms_data['SentDate'] > delivered_sent_date)
    ]

    if not matching_responses.empty:
        # Loop through all responses
        for _, response in matching_responses.iterrows():
            qa_responses.append({
                'Phone Number': question_to,
                'Question': question_body,
                'Answer': response['Body'],
                'Question Sent Date': delivered_sent_date,
                'Answer Sent Date': response['SentDate']
            })
            break  # Only capture the first valid response for each question
    else:
        # If no answer is found, record "No Response"
        qa_responses.append({
            'Phone Number': question_to,
            'Question': question_body,
            'Answer': "No Response",
            'Question Sent Date': delivered_sent_date,
            'Answer Sent Date': None
        })

# Convert results to a DataFrame
qa_mapping_sorted = pd.DataFrame(qa_responses)

# Save the sorted and processed data to a CSV file
output_file_path = 'updated_sorted_question_answer_pairs.csv'
qa_mapping_sorted.to_csv(output_file_path, index=False)

# Print confirmation
print(f"Updated sorted question-answer mappings saved to {output_file_path}")

# Display the DataFrame
print(qa_mapping_sorted)


<ipython-input-32-43752e33ca1e>:18: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  delivered_sms_data['SentDate'] = pd.to_datetime(delivered_sms_data['SentDate'], errors='coerce')
<ipython-input-32-43752e33ca1e>:19: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  received_sms_data['SentDate'] = pd.to_datetime(received_sms_data['SentDate'], errors='coerce')


Updated sorted question-answer mappings saved to updated_sorted_question_answer_pairs.csv
      Phone Number                                           Question  \
0     1.860330e+10  Hi! My name is Helius🙂 I'm excited to hear tha...   
1     1.860777e+10  Hi! My name is Helius🙂 I'm excited to hear tha...   
2     1.860287e+10  Hi! My name is Helius🙂 I'm excited to hear tha...   
3     1.860606e+10  Hi! My name is Helius🙂 I'm excited to hear tha...   
4     1.860287e+10                        Ok, what's your full name ?   
...            ...                                                ...   
1937  1.404934e+10  If offered a job, how soon can you start?\r\n1...   
1938  1.404934e+10  Perfect, \r\n\r\nClick here: https://helius.cl...   
1939  1.914883e+10  Hi! My name is Helius🙂, and I'm excited to sho...   
1940  1.914883e+10                           And, what's your email ?   
1941  1.914883e+10  Now let's get to know you.\r\n\r\nAre a licens...   

                 Answer         Q

In [ ]:
json_data = qa_mapping_sorted.to_json()

Upload .env File and get the SUPABASE environment variables

In [ ]:
from google.colab import files
uploaded = files.upload()
!pip install python-dotenv

Saving .env to .env


Step 2 - Use Parsed Sorted File to populate the Database

In [ ]:
# Load .env file
load_dotenv('.env')

# Access environment variables
SUPABASE_CANDIDATE_URL = os.getenv('SUPABASE_CANDIDATE_URL')
SUPABASE_PROJECT_URL = os.getenv('SUPABASE_PROJECT_URL')
SUPABASE_API_KEY = os.getenv('SUPABASE_API_KEY')
SUPABASE_SERVICE_KEY = os.getenv('SUPABASE_SERVICE_KEY')

supabase: Client = create_client(SUPABASE_PROJECT_URL, SUPABASE_SERVICE_KEY)

In [ ]:
from supabase import create_client
from supabase.lib.client_options import ClientOptions

supabase = create_client(
  SUPABASE_PROJECT_URL,
  SUPABASE_SERVICE_KEY,
  options=ClientOptions(
    auto_refresh_token=False,
    persist_session=False,
  )
)

# Access auth admin api
admin_auth_client = supabase.auth.admin
user_list = supabase.auth.admin.list_users(1, 4000)




Helper Functions

In [ ]:
def get_user_id_by_email(email: str):
    try:
        for user in user_list:
            if user.email == email:
              return user.id
        return None
    except Exception as e:
        print(f"Error fetching user by email: {e}")
        return None

In [ ]:
def check_user_exists(email: str):
    try:
        for user in user_list:
            if user.email == email:
              return True
        return False
    except Exception as e:
        print(f"Error fetching user by email: {e}")
        return False

In [ ]:
def format_phone_number(phone_number):
    # Ensure the phone number is an integer and then convert to string
    phone_number = str(int(phone_number))

    # Format the phone number
    formatted_number = f"({phone_number[1:4]}) {phone_number[4:7]}-{phone_number[7:]}"
    return formatted_number

In [ ]:
# Function to split full_name into first_name and last_name
def split_full_name(full_name):
    name_parts = full_name.strip().split()
    first_name = name_parts[0] if name_parts else ''
    last_name = ' '.join(name_parts[1:]) if len(name_parts) > 1 else ''
    return first_name, last_name

In [ ]:
def get_user_id_by_phone(phone: str):
  try:
        # Query Supabase for the candidate ID based on phone number
        response = (
            supabase.table("hel_candidate")
            .select("candidate_id")
            .filter("phone_number", "eq", phone)
            .execute()
        )

        # Check if there is an error in the response
        if response.data and len(response.data) > 0:
            return response.data[0]["candidate_id"]
        else:
           return None

  except Exception as e:
        print(f"Exception while fetching candidate ID: {e}")
        return None

In [ ]:
user_id = get_user_id_by_phone(format_phone_number(19148430241.0))
print("User ID = ",user_id)

Exception while fetching candidate ID: name 'supabase' is not defined
User ID =  None


In [ ]:
# Headers for admin access
headers = {
    "Authorization": f"Bearer {SUPABASE_SERVICE_KEY}",
    "apikey": SUPABASE_SERVICE_KEY,
    "Content-Type": "application/json",
}

# Function to create or update (upsert) a user
def upsert_user(email, password):
    existing_user = check_user_exists(email)
    if existing_user:
        # Update logic (optional; Supabase admin API doesn't allow updates directly)
        print(f"User already exists: {email}. Skipping creation.")
    else:
        # Create new user
        data = {
            "email": email,
            "password": password,
            "email_confirm": True  # Automatically confirm the email
        }
        response = requests.post(
            f"{SUPABASE_PROJECT_URL}/auth/v1/admin/users", headers=headers, json=data
        )
        if response.status_code in [200, 201]:
            print(f"User created successfully: {email}")
        else:
            print(f"Failed to create user {email}: {response.text}")


Start with Email Address. Populate in the Hel_Candidate Table

In [ ]:
# Filter the DataFrame - Get all Email Addresses and remove No Response answers
filtered_df = qa_mapping_sorted[
    ((qa_mapping_sorted["Question"] == "And, what's your email ?") |
     (qa_mapping_sorted["Question"] == "Oops, the email you entered doesn't not matched. Please try again.")) &
    (qa_mapping_sorted["Answer"] != "No Response")
]

# Upsert data into the Supabase table
for index, row in filtered_df.iterrows():
    phone_number = format_phone_number(row["Phone Number"])
    email = row["Answer"].lower()

    #Create user if it doesnt exist using the email Address
    if not check_user_exists(email):
      upsert_user(email, "test1234")

    # Upsert data into the Candidate Table if an id was created
    if get_user_id_by_email(email) is not None:
      data = {
          "candidate_id": get_user_id_by_email(email),  # Generate a new UUID for each candidate
          "email": email,   # Email
          "phone_number": phone_number,   # Phone Number
          "creation_date": datetime.now().date().isoformat(),  # Use current date
          "created_by": "ADMIN",
          "last_update_date": datetime.now().date().isoformat(),
          "last_updated_by": "ADMIN"
      }
      print(data)

      response = (
              supabase.table("hel_candidate")
               .upsert(data)
               .execute()
           )

      # Check if there is an error in the response
      if response:
          print(f"Successfully uploaded: {data['email']}")
      else:
          print(f"Failed to upload: {data['email']} - {response.error}")



{'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'email': 'kaitlynnichols22@gmail.com', 'phone_number': '(860) 287-1220', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: kaitlynnichols22@gmail.com
{'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'email': 'mariapedraza1010@gmail.com', 'phone_number': '(860) 605-5438', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: mariapedraza1010@gmail.com
{'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'email': 'markrennsagun@yahoo.com', 'phone_number': '(860) 823-8972', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: markrennsagun@yahoo.com
Failed to create user joelfrancois.jr@gmail.com : {"code":400,"error_code":"validation_failed","msg":"Unable to valida

Function to load other Fields based on the question

In [ ]:
def populate_helius_table(qa_mapping_sorted, question, table_name, field_name):
    # Filter the DataFrame - Get all Questions and remove No Response answers
    # Normalize the input question
    question_cleaned = question.strip().replace("\n", " ").replace("\t", " ").replace("\r", " ")

    # Normalize the Question column in the DataFrame for accurate matching
    qa_mapping_sorted["Question"] = qa_mapping_sorted["Question"].str.strip().str.replace("\n", " ").str.replace("\t", " ").str.replace("\r", " ")

    # Normalize the Question column in the DataFrame
    qa_mapping_sorted["Question"] = qa_mapping_sorted["Question"].str.strip().str.replace("\n", " ").str.replace("\t", " ")

    # Filter using .str.contains() to match partial text
    filtered_df = qa_mapping_sorted[
        qa_mapping_sorted["Question"].str.contains(question_cleaned, case=False, na=False) &
        (qa_mapping_sorted["Answer"] != "No Response")
    ]


    # Upsert data into the Supabase table
    for index, row in filtered_df.iterrows():
        phone_number = row["Phone Number"]
        phone_number = format_phone_number(phone_number)
        answer = row["Answer"].lower()

        # Upsert data into the Candidate Table if an id was created
        if get_user_id_by_phone(phone_number) is not None:
          if(field_name == "full_name"):
            first_name, last_name = split_full_name(answer)
            data = {
                "phone_number": phone_number,
                "candidate_id": get_user_id_by_phone(phone_number),  # Generate a new UUID for each candidate
                "first_name": first_name,   # Field Name
                "last_name": last_name,   # Field Name
                "creation_date": datetime.now().date().isoformat(),  # Use current date
                "created_by": "ADMIN",
                "last_update_date": datetime.now().date().isoformat(),
                "last_updated_by": "ADMIN"
            }
          elif(table_name != "hel_candidate"):
            data = {
                "candidate_id": get_user_id_by_phone(phone_number),  # Generate a new UUID for each candidate
                field_name: answer,   # Field Name
                "creation_date": datetime.now().date().isoformat(),  # Use current date
                "created_by": "ADMIN",
                "last_update_date": datetime.now().date().isoformat(),
                "last_updated_by": "ADMIN"
            }
          else:
            data = {
                "phone_number": phone_number,
                "candidate_id": get_user_id_by_phone(phone_number),  # Generate a new UUID for each candidate
                field_name: answer,   # Field Name
                "creation_date": datetime.now().date().isoformat(),  # Use current date
                "created_by": "ADMIN",
                "last_update_date": datetime.now().date().isoformat(),
                "last_updated_by": "ADMIN"
            }
          print(data)

          response = (
                  supabase.table(table_name)
                  .upsert(data)
                  .execute()
              )

          # Check if there is an error in the response
          if response:
              print(f"Successfully uploaded: {data['candidate_id']}")
          else:
              print(f"Failed to upload: {data['candidate_id']} - {response.error}")


Populate Full Name

In [ ]:
populate_helius_table(qa_mapping_sorted, "Ok, what's your full name ?", "hel_candidate", "full_name")

{'phone_number': '(860) 287-1220', 'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'first_name': 'kaitlyn', 'last_name': 'nichols', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'phone_number': '(860) 605-5438', 'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'first_name': 'maria', 'last_name': 'pedraza', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'phone_number': '(860) 823-8972', 'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'first_name': 'mark', 'last_name': 'renn sagun', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02
{'phone_number': '(203) 917-5593', 'candidate_id': '43

In [ ]:
populate_helius_table(qa_mapping_sorted, "Please enter your first and last name.", "hel_candidate", "full_name")

{'phone_number': '(660) 498-5863', 'candidate_id': 'bdd2c34a-1f79-477f-8029-1f14eddbc798', 'first_name': 'david', 'last_name': 'johnson', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: bdd2c34a-1f79-477f-8029-1f14eddbc798


Populate Needs Visa Sponsorship

In [ ]:
populate_helius_table(qa_mapping_sorted, "Will you now or in the future need visa sponsorship?", "hel_candidate", "visa_sponsorship_required")

{'phone_number': '(860) 287-1220', 'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'visa_sponsorship_required': 'no', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'phone_number': '(860) 605-5438', 'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'visa_sponsorship_required': '2', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'phone_number': '(860) 823-8972', 'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'visa_sponsorship_required': 'no', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02


Populate Preferred Position

In [ ]:
populate_helius_table(qa_mapping_sorted, "What kind of position are you interested in?", "hel_preferences", "preference_position")

{'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'preference_position': 'full time', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'preference_position': '1', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'preference_position': 'internship', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02


Populate Graduated Flag

In [ ]:
populate_helius_table(qa_mapping_sorted, "Have you graduated?", "hel_candidate", "graduated_flag")

{'phone_number': '(860) 287-1220', 'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'graduated_flag': 'no', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'phone_number': '(860) 605-5438', 'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'graduated_flag': '2', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'phone_number': '(860) 823-8972', 'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'graduated_flag': 'no', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02


Populate Graduation Year

In [ ]:
populate_helius_table(qa_mapping_sorted, "What year are you graduating?", "hel_education", "graduation_year")

{'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'graduation_year': 'may-24', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'graduation_year': '2024', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'graduation_year': '2024', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02
{'candidate_id': '96a28534-88dc-4dc9-9478-a1e0c234d421', 'graduation_year': '2024', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploa

Populate Major - Field of Study




In [ ]:
populate_helius_table(qa_mapping_sorted, "What is / was your major?", "hel_education", "field_of_study")

{'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'field_of_study': 'nursing ', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'field_of_study': 'nursing', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'field_of_study': 'human services', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02
{'candidate_id': '96a28534-88dc-4dc9-9478-a1e0c234d421', 'field_of_study': 'adn', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successf

Populate Helius Extra Curricular data

In [ ]:
populate_helius_table(qa_mapping_sorted, "Were you part of any extra-curricular groups, clubs, or activities?", "hel_extra_curricular", "extra_curricular_name")

{'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'extra_curricular_name': 'no', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'extra_curricular_name': '1', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'extra_curricular_name': 'no', 'creation_date': '2025-02-01', 'created_by': 'ADMIN', 'last_update_date': '2025-02-01', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02


Populate Licensed Nurse data

In [ ]:
populate_helius_table(qa_mapping_sorted, "Are a licensed Nurse or still a Student ?", "hel_candidate", "licensed_nurse")

{'phone_number': '(860) 287-1220', 'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'licensed_nurse': '2', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'phone_number': '(860) 605-5438', 'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'licensed_nurse': 'student nurse', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'phone_number': '(860) 823-8972', 'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'licensed_nurse': 'student', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02
{'phone_number': '(660) 498-5863', 'candidate_id': 'bdd2c34a-1f79-477f-8029-1f14eddbc798', 'licensed_nurse': 'hi

Populate Educational Achievements

In [ ]:
populate_helius_table(qa_mapping_sorted, "Were you honored with any awards that you will note on your resume (honor society, academic awards, etc.)?", "hel_education", "educational_achievements")

<ipython-input-20-29bd4356f6cf>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  qa_mapping_sorted["Question"].str.contains(question_cleaned, case=False, na=False) &


{'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'educational_achievements': 'honor society ', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'educational_achievements': 'honors ', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02
{'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'educational_achievements': 'vice president of national honors society\r\nleadership award', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59


Populate State of Residency

In [ ]:
populate_helius_table(qa_mapping_sorted, "which state do you live in?", "hel_candidate", "state_of_residency")

{'phone_number': '(860) 287-1220', 'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'state_of_residency': 'ct', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'phone_number': '(860) 823-8972', 'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'state_of_residency': 'ct', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02
{'phone_number': '(860) 605-5438', 'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'state_of_residency': 'ct', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'phone_number': '(860) 878-1767', 'candidate_id': '96a28534-88dc-4dc9-9478-a1e0c234d421', 'state_of_residency': 'c

Populate Willing to Relocate data

In [ ]:
populate_helius_table(qa_mapping_sorted, "Are you willing to relocate for a position?", "hel_candidate", "willing_to_relocate")

{'phone_number': '(860) 287-1220', 'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'willing_to_relocate': 'yes', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'phone_number': '(860) 823-8972', 'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'willing_to_relocate': 'no', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02
{'phone_number': '(860) 605-5438', 'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'willing_to_relocate': '2', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'phone_number': '(860) 878-1767', 'candidate_id': '96a28534-88dc-4dc9-9478-a1e0c234d421', 'willing_to_relocate'

Populate Goals

In [ ]:
populate_helius_table(qa_mapping_sorted, "What are your goals in Nursing?", "hel_candidate", "goals")

{'phone_number': '(860) 605-5438', 'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'goals': 'student nurse', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'phone_number': '(860) 823-8972', 'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'goals': 'care for patients', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02


Populate Start Availability

In [ ]:
populate_helius_table(qa_mapping_sorted, "If offered a job, how soon can you start?", "hel_candidate", "start_availability")

{'phone_number': '(860) 287-1220', 'candidate_id': 'e45a1965-2d38-4b0c-b172-cefdfd0b8aa8', 'start_availability': '1', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e45a1965-2d38-4b0c-b172-cefdfd0b8aa8
{'phone_number': '(860) 823-8972', 'candidate_id': '407572fc-0914-4feb-8c70-83cfffe69a02', 'start_availability': '3', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 407572fc-0914-4feb-8c70-83cfffe69a02
{'phone_number': '(860) 605-5438', 'candidate_id': 'fbe157a8-361e-4894-a051-b8ceec08eb59', 'start_availability': '3', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: fbe157a8-361e-4894-a051-b8ceec08eb59
{'phone_number': '(860) 878-1767', 'candidate_id': '96a28534-88dc-4dc9-9478-a1e0c234d421', 'start_availability': '1', 